# Carbon Use Efficiency

Resources:
* Models: http://bigg.ucsd.edu/models/

In [ ]:
import requests, cobra, tempfile

In [1]:
from cue_tools import loadModelURL

In [ ]:
def loadModelURL(url):
    """Load SMBL cobra model from a URL"""
    with tempfile.NamedTemporaryFile('w') as tf:
        handle = requests.get(url)
        tf.write(handle.text)
        model = cobra.io.read_sbml_model(tf.name)
        return model

# Load

In [ ]:
"""Load model"""
url = 'http://bigg.ucsd.edu/static/models/e_coli_core.xml'
model = loadModelURL(url)
model

Name,e_coli_core
Memory address,0x07efd219fde10
Number of metabolites,72
Number of reactions,95
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [31]:
res = model.optimize()
